In [226]:
from keras.callbacks import Callback
import keras.backend as K
import numpy as np
from keras_preprocessing.image import ImageDataGenerator
import librosa
import librosa.display
import os
import errno
import matplotlib.pyplot as plt
from matplotlib import figure
from PIL import Image

In [224]:
#Creating the required folders
try:
    os.makedirs("image/train/cat")
    os.makedirs("image/train/dog")
    os.makedirs("image/test/cat")
    os.makedirs("image/test/dog")
except FileExistsError:
    # directory already exists
    pass

In [16]:
#Function for creating the spectrogram of the audio from train dataset
def create_spectrogram(filename,name,folder):
    plt.interactive(False)
    clip, sample_rate = librosa.load('cats_dogs/train/'+folder+'/'+filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = 'image/train/'+ folder +'/' + name + '.jpg'
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')

In [17]:
#Fucntion for creating the spectrogram of the audio from test dataset
def create_spectrogram_test(filename,name,folder):
    plt.interactive(False)
    clip, sample_rate = librosa.load('cats_dogs/test/'+folder+'/'+filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = 'image/test/'+ folder +'/' + name + '.jpg'
    fig.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')

In [15]:
#Creating the spectrogram of the train split

a=0
for subdir in os.listdir('cats_dogs/train'):
  for filename in os.listdir('cats_dogs/train/'+subdir):
    filename,name = filename,filename.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name,subdir)
    print("Image",a,'created')
    a=a+1

image  0 created
image  1 created
image  2 created
image  3 created
image  4 created
image  5 created
image  6 created
image  7 created
image  8 created
image  9 created
image  10 created
image  11 created
image  12 created
image  13 created
image  14 created
image  15 created
image  16 created
image  17 created
image  18 created
image  19 created
image  20 created
image  21 created
image  22 created
image  23 created
image  24 created
image  25 created
image  26 created
image  27 created
image  28 created
image  29 created
image  30 created
image  31 created
image  32 created
image  33 created
image  34 created
image  35 created
image  36 created
image  37 created
image  38 created
image  39 created
image  40 created
image  41 created
image  42 created
image  43 created
image  44 created
image  45 created
image  46 created
image  47 created
image  48 created
image  49 created
image  50 created
image  51 created
image  52 created
image  53 created
image  54 created
image  55 created
im

In [228]:

#Renaming the test folders
try:
    os.rename('cats_dogs/test/cats','cats_dogs/test/cat')
    os.rename('cats_dogs/test/test','cats_dogs/test/dog')
except FileNotFoundError:
    # directory name already changed
    pass
a=0
for subdir in os.listdir('cats_dogs/test'):
  for filename in os.listdir('cats_dogs/test/'+subdir):
    filename,name = filename,filename.split('/')[-1].split('.')[0]
    create_spectrogram_test(filename,name,subdir)
    print("Image",a,'created')
    a=a+1

Image 0 created
Image 1 created
Image 2 created
Image 3 created
Image 4 created
Image 5 created
Image 6 created
Image 7 created
Image 8 created
Image 9 created
Image 10 created
Image 11 created
Image 12 created
Image 13 created
Image 14 created
Image 15 created
Image 16 created
Image 17 created
Image 18 created
Image 19 created
Image 20 created
Image 21 created
Image 22 created
Image 23 created
Image 24 created
Image 25 created
Image 26 created
Image 27 created
Image 28 created
Image 29 created
Image 30 created
Image 31 created
Image 32 created
Image 33 created
Image 34 created
Image 35 created
Image 36 created
Image 37 created
Image 38 created
Image 39 created
Image 40 created
Image 41 created
Image 42 created
Image 43 created
Image 44 created
Image 45 created
Image 46 created
Image 47 created
Image 48 created
Image 49 created
Image 50 created
Image 51 created
Image 52 created
Image 53 created
Image 54 created
Image 55 created
Image 56 created
Image 57 created
Image 58 created
Image 5

In [66]:
training_datagen=ImageDataGenerator(rescale=1./255.)

validation_datagen=ImageDataGenerator(rescale=1./255)

train_generator=training_datagen.flow_from_directory(
    directory='image/train',
    class_mode='categorical',
    shuffle=True,
    seed=37,
    target_size=(128,128),
    batch_size=32
)

validation_generator=validation_datagen.flow_from_directory(
    directory='image/test',
    class_mode='categorical',
    shuffle=True,
    seed=37,
    target_size=(128,128),
    batch_size=32
)

Found 210 images belonging to 2 classes.
Found 67 images belonging to 2 classes.


In [188]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Flatten,Dropout,Conv2D, MaxPooling2D,Activation,LSTM

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(128,128,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [68]:
#Callback fucntion to stop training when validation accuracy  reaches 95%
class myCallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_acc')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

model.compile(optimizers.Adam(lr=1e-4, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [72]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
callbacks = myCallback()

History=model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,
                    callbacks=[callbacks]
)

Epoch 1/50
6/6 [==============================] - 1s 177ms/step - loss: 0.1983 - acc: 0.9326 - val_loss: 0.2776 - val_acc: 0.9062
Epoch 2/50
6/6 [==============================] - 1s 130ms/step - loss: 0.1725 - acc: 0.9438 - val_loss: 0.2608 - val_acc: 0.9062
Epoch 3/50
6/6 [==============================] - 1s 130ms/step - loss: 0.1761 - acc: 0.9531 - val_loss: 0.2548 - val_acc: 0.9375
Epoch 4/50
5/6 [========================>.....] - ETA: 0s - loss: 0.1604 - acc: 0.9394
Reached 95% accuracy so cancelling training!
6/6 [==============================] - 1s 120ms/step - loss: 0.1549 - acc: 0.9451 - val_loss: 0.2443 - val_acc: 0.9531


In [70]:
#Function for creating spectrogram for the input audio to be predicted
def create_spectrogram_pred(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  =  name + '.jpg'
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [77]:
#Function for predicting cat or dog of the input audio
def pred_sound(filename):
  filename,name = filename,filename.split('/')[-1].split('.')[0]
  create_spectrogram_pred(filename,name)
  img = Image.open(name+'.jpg')
  img = img.resize((128,128))
  sample=np.expand_dims(img,axis=0)
  y_hat=model.predict(sample)
  if np.argmax(y_hat[0])==0:
    print("The audio is by cat")
  else:
    print("The audio is by dog")


In [78]:
pred_sound('bark.wav')

The audio is by dog
